# Using equilibrated LJ clusters from Wales database to compute binding energies

2024-05-11

__Process:__
- DONE download data (cluster coordinates) from Wales database up to 240 nucleons
- DONE for each cluster, compute lattice constant and rescale so that it becomes 1 fm (bottom of strong force LJ well)
- DONE part 1: for a range of proton numbers, randomly assign particle types and compute total (binding) energy
- DONE part 2: perform random assignment several times (100s), recompute energies
- DONE part 3-1: equilibrate selected atoms lightly using basin hopping before energy computation
- part 3-2: equilibrate all atoms using gradient descent, recompute the binding energy curve
- DONE part 4: try Morse potential, fine-tune force field parameters to match the binding energy curve

__Data source:__
- entry: http://doye.chem.ox.ac.uk/jon/structures/LJ.html
- 3-150: http://doye.chem.ox.ac.uk/jon/structures/LJ/tables.150.html
- 151-309: https://chinfo.nankai.edu.cn/chmm/pubmats/LJ/ljstructures_e.html
- 310-561: http://doye.chem.ox.ac.uk/jon/structures/LJ/LJ310-561.html
- 562-1000: http://doye.chem.ox.ac.uk/jon/structures/LJ/LJ562-1000.html

Coordimates downloaded from zipped files on the websites.

__Results:__
- LJ potential works if eps is around 3.2 (without equilibration), not 50 MeV, as previously assumed (source: Imperial lecture notes)
- LJ potential does not reproduce reliably the binding energy curve, towards higher A the binding energy is still rather flat; a more short-term attractive interaction is needed
- BUT: LJ potential could be better at reproducing if sigma is also varied; it should get smaller to lift the binding energy
- sampling nucleon types (permuting protons and neutrons) creates periodic patterns on the scale of A ~ 20 (when energy is computed without equilibration)
- from basin hopping results it seems that the descent to the already found global LJ minima is the global minimum for morse potential; this suggests there is some kind of topology at play; hence, a simple gradient descent should suffice to find the global minimum for a Morse cluster by descending straight to it

__Next steps beyond this notebook:__
- install LAMMPS to simulate the dynamics (neutron decay) as well as reproduce finding the minima

In [6]:
import numpy as np
import jax
import jax.numpy as jnp
import pandas as pd

import plotly.express as px

from datetime import datetime

pd.options.plotting.backend = 'plotly'

In [7]:
from functions_io import read_coords

In [8]:
eps0 = 8.85418782e-12
e = 1.60217662e-19
c = 299792458.0

# reduced units
rc = 1e-15
ec = 1.60217662e-19
mc = 1.6726219e-27

## Define functions

In [81]:
# energy scales

# ideal LJ parameters
EPS_STRONG = 3.2

# ideal morse parameters
# EPS_STRONG = 3.5
ALPHA_STRONG = 8.0

In [82]:
def elmag_potential(r, t1, t2):
    """In MeV, r in fm"""
    return 1.44 * t1 * t2 / r

def lj_potential(r, epsilon=EPS_STRONG, sigma=1/2**(1/6)):
    """In MeV, r in fm"""
    return 4 * epsilon * ((sigma / r) ** 12 - (sigma / r) ** 6)

def morse_potential(r, D=EPS_STRONG, alpha=ALPHA_STRONG, re=1.0):
    """In MeV, r in fm"""
    return D * ((1.0 - np.exp(-alpha * (r - re))) ** 2 - 1.0)

In [11]:
def total_energy(R, T, pot='lj', mode='bh'):
    if mode == 'bh':
        R = R.reshape(-1, 3)
    Ve, Vs= 0.0, 0.0
    for i, _ in enumerate(T):
        for j in range(i):
            r = np.linalg.norm(R[i] - R[j])
            Ve += elmag_potential(r, T[i], T[j])
            if pot == 'lj':
                Vs += lj_potential(r)
            elif pot == 'morse':
                Vs += morse_potential(r)
    V = Ve + Vs
    return V, Ve, Vs

def total_energy_min(R, T, pot='lj', mode='bh'):
    '''Same as total energy, only returning one value for minimisation'''
    if mode == 'bh':
        R = R.reshape(-1, 3)
    Ve, Vs= 0.0, 0.0
    for i, _ in enumerate(T):
        for j in range(i):
            r = np.linalg.norm(R[i] - R[j])
            Ve += elmag_potential(r, T[i], T[j])
            if pot == 'lj':
                Vs += lj_potential(r)
            elif pot == 'morse':
                Vs += morse_potential(r)
    V = Ve + Vs
    return V

def total_energy_jnp(R, T, pot='lj'):
    Ve, Vs = 0.0, 0.0
    for i, _ in enumerate(T):
        for j in range(i):
            r = jnp.linalg.norm(R[i] - R[j])
            Ve += elmag_potential(r, T[i], T[j])
            if pot == 'lj':
                Vs += lj_potential(r)
            elif pot == 'morse':
                Vs += morse_potential(r)
    V = Ve + Vs
    return V, Ve, Vs

def minimize_energy(R, T, n_steps=10, lr=1e-5, verbose=False, thermo=10, pot='lj'):
    """Gradient descent for minimisation"""
    for i in range(n_steps):
        R -= lr * jax.grad(total_energy_jnp)(R, T)
        if verbose and i % thermo == 0:
            print(i, total_energy_jnp(R, T, pot=pot))[0]
    return R

In [12]:
def generate_random_labels(Z, N):
    T = np.array([1] * Z + [0] * N)
    np.random.shuffle(T)
    return T

In [13]:
def distance_vector(X):
    N = X.shape[0]
    D = []
    for i in range(N):
        for j in range(i):
            D.append(np.linalg.norm(X[i] - X[j]))
    return np.array(sorted(D))

## Collect proton and neutron numbers

In [35]:
dfelem = pd.read_csv('elements.csv')
dfelem.columns = ['element', 'symbol', 'Z', 'N', 'A']
dfelem = dfelem.set_index('element')

In [15]:
dfelem.head()

,symbol,Z,N,A
element,,,,
Hydrogen,H,1,0,1
Helium,He,2,2,4
Lithium,Li,3,4,7
Beryllium,Be,4,5,9
Boron,B,5,6,11


## Evaluate distances and energies

In [16]:
Z = 20
A = 25
N = A - Z

In [17]:
X = read_coords(A)
T = generate_random_labels(N, Z)

In [19]:
distance_vector(X)[:10]

array([1.04836005, 1.04836741, 1.05277987, 1.05969946, 1.05970247,
       1.06264336, 1.06264689, 1.06266104, 1.06266332, 1.06350952])

In [20]:
total_energy_jnp(X, T)

(Array(-261.58084, dtype=float32),
 Array(7.147557, dtype=float32),
 Array(-268.7284, dtype=float32))

## Part 1: compute energies for a range of elements

### Using LJ potential

In [83]:
np.random.seed(42)

In [84]:
list_results = []

In [105]:
%%time

for elem in dfelem.index[1:94]:
    ti = datetime.now()
    symbol, N, Z, A = dfelem.loc[elem]
    # print(symbol, N, Z, A)

    # read coordinates    
    X = read_coords(A)
    T = generate_random_labels(N, Z)

    # compute energy, elmag and strong contribution
    E, Ee, Es = total_energy(X, T)
    epn = E / A
    print(elem, N, A, E, epn)
    # print(f'Time', datetime.now() - ti)
    
    list_results.append(
        {'element': elem, 'N': N, 'A': A, 'E': E, 'Ee': Ee, 'Es': Es, 'epn': epn}
    )

Helium 2 4 -13.117147755763861 -3.2792869389409653
Lithium 3 7 -36.23871490136544 -5.176959271623635
Beryllium 4 9 -51.18487641483363 -5.687208490537071
Boron 5 11 -67.71204703719421 -6.155640639744928
Carbon 6 12 -76.75097915397821 -6.395914929498184
Nitrogen 7 14 -94.38810763945739 -6.742007688532671
Oxygen 8 16 -109.31038262162335 -6.831898913851459
Fluorine 9 19 -139.787933226348 -7.357259643492
Neon 10 20 -142.9975356837036 -7.14987678418518
Sodium 11 23 -175.4073916178845 -7.62640833121237
Magnesium 12 24 -174.54477667830426 -7.272699028262678
Aluminum 13 27 -201.66544782509993 -7.469090660188886
Silicon 14 28 -207.06012417267073 -7.39500443473824
Phosphorus 15 31 -240.9705106802519 -7.773242280008126
Sulfur 16 32 -240.08015468583955 -7.502504833932486
Chlorine 17 35 -268.9618662193703 -7.684624749124865
Argon 18 40 -326.4139958391018 -8.160349895977545
Potassium 19 39 -302.5618395885193 -7.75799588688511
Calcium 20 40 -300.85113113910154 -7.5212782784775385
Scandium 21 45 -365.9

### Process and visualise data

In [86]:
dfres = pd.DataFrame(list_results)
dfres['e_elmag_pn'] = dfres['Ee'] / dfres['A']
dfres['e_strong_pn'] = dfres['Es'] / dfres['A']
dfres.head()

,element,N,A,E,Ee,Es,epn,e_elmag_pn,e_strong_pn
0,Helium,2,4,-13.117148,1.282895,-14.400043,-3.279287,0.320724,-3.600011
1,Lithium,3,7,-36.238715,3.374234,-39.612949,-5.176959,0.482033,-5.658993
2,Beryllium,4,9,-51.184876,6.687175,-57.872052,-5.687208,0.743019,-6.430228
3,Boron,5,11,-67.712047,10.926475,-78.638522,-6.155641,0.993316,-7.148957
4,Carbon,6,12,-76.750979,14.371145,-91.122124,-6.395915,1.197595,-7.593510


In [96]:
dfshow = dfres.set_index('A')[['epn', 'e_elmag_pn', 'e_strong_pn']].unstack().reset_index()
dfshow.columns = ['component', 'A', 'value']
dfshow.tail()

,component,A,value
274,e_strong_pn,232,-15.023414
275,e_strong_pn,231,-15.021413
276,e_strong_pn,238,-15.069965
277,e_strong_pn,237,-15.060621
278,e_strong_pn,244,-15.116880


In [101]:
px.scatter(dfres, x='A', y='epn', title='Energy per nucleon vs number of nucleons', range_y=[-10, 0])

In [98]:
px.scatter(dfshow, x='A', y='value', color='component')

In [90]:
# save data gradually

# dfres.to_csv('run_lj_no_eq.csv', index=False)
# dfres.to_csv('run_eq.csv', index=False)
# dfres.to_csv('run_1_no_eq.csv', index=False)

### Reproduce results with Morse potential

In [71]:
np.random.seed(42)

In [72]:
list_results = []

In [74]:
%%time

for elem in dfelem.index[1:94]:
    ti = datetime.now()
    symbol, N, Z, A = dfelem.loc[elem]
    # print(symbol, N, Z, A)

    # read coordinates    
    X = read_coords(A)
    T = generate_random_labels(N, Z)

    # compute energy, elmag and strong contribution
    E, Ee, Es = total_energy(X, T, pot='morse')
    epn = E / A
    print(elem, N, A, E, epn)
    # print(f'Time', datetime.now() - ti)
    
    list_results.append(
        {'element': elem, 'N': N, 'A': A, 'E': E, 'Ee': Ee, 'Es': Es, 'epn': epn}
    )

Helium 2 4 -11.525201312691332 -2.881300328172833
Lithium 3 7 -30.882687942551378 -4.411812563221625
Beryllium 4 9 -44.11853684444634 -4.9020596493829265
Boron 5 11 -58.46420269465246 -5.314927517695678
Carbon 6 12 -63.11875717314649 -5.259896431095541
Nitrogen 7 14 -77.62679151317667 -5.544770822369762
Oxygen 8 16 -90.6067495729042 -5.662921848306513
Fluorine 9 19 -115.6189064043693 -6.085205600229963
Neon 10 20 -109.18235515392172 -5.459117757696086
Sodium 11 23 -138.07100500908905 -6.003087174308219
Magnesium 12 24 -137.01349313262665 -5.708895547192777
Aluminum 13 27 -154.12255216027634 -5.708242672602828
Silicon 14 28 -158.44686110933742 -5.658816468190622
Phosphorus 15 31 -182.89632928526942 -5.899881589847401
Sulfur 16 32 -181.85223874109545 -5.682882460659233
Chlorine 17 35 -198.52533855092878 -5.672152530026537
Argon 18 40 -252.04489261620367 -6.301122315405092
Potassium 19 39 -229.96809511317278 -5.896617823414687
Calcium 20 40 -226.118677933829 -5.652966948345725
Scandium 21

Chromium 24 52 -309.4704892878882 -5.951355563228619
Manganese 25 55 -338.7136392804325 -6.158429805098773
Iron 26 56 -327.9976627479956 -5.857101120499921
Cobalt 27 59 -359.3647547669077 -6.090928046896741
Nickel 28 59 -327.0421957612011 -5.543088063749171
Copper 29 64 -377.3367349033756 -5.895886482865244
Zinc 30 65 -369.78820273298004 -5.689049272815078
Gallium 31 70 -411.14228714497904 -5.873461244928272
Germanium 32 73 -433.73243789308515 -5.9415402451107555
Arsenic 33 75 -448.0275725415911 -5.973700967221214
Selenium 34 79 -480.28624510027845 -6.079572722788335
Bromine 35 80 -455.80930445894455 -5.6976163057368066
Krypton 36 84 -493.1751682025893 -5.87113295479273
Rubidium 37 85 -483.2295569439787 -5.685053611105632
Strontium 38 88 -496.00890549732 -5.636464835196818
Yttrium 39 89 -497.14643670334647 -5.585915019138724
Zirconium 40 91 -516.8051202101182 -5.6791771451661335
Niobium 41 93 -496.85553596360705 -5.342532644769968
Molybdenum 42 96 -504.2743335073054 -5.252857640701098


In [75]:
dfres = pd.DataFrame(list_results)
dfres['e_elmag_pn'] = dfres['Ee'] / dfres['A']
dfres['e_strong_pn'] = dfres['Es'] / dfres['A']
dfres.tail()

,element,N,A,E,Ee,Es,epn,e_elmag_pn,e_strong_pn
162,Thorium,90,232,-982.955088,1782.679676,-2765.634764,-4.236875,7.683964,-11.920839
163,Protactinium,91,231,-899.976340,1853.315030,-2753.291370,-3.896001,8.023009,-11.919010
164,Uranium,92,238,-955.312621,1900.975327,-2856.287948,-4.013919,7.987291,-12.001210
165,Neptunium,93,237,-863.107156,1979.003726,-2842.110882,-3.641802,8.350227,-11.992029
166,Plutonium,94,244,-953.540585,1970.187598,-2923.728183,-3.907953,8.074539,-11.982493


In [76]:
# reshape the dataframe
dfshow = dfres.melt(id_vars=['A'], value_vars=['epn', 'e_elmag_pn', 'e_strong_pn'], var_name='component', value_name='value')

In [77]:
px.scatter(dfres, x='A', y='epn', title='Energy per nucleon vs number of nucleons')

In [63]:
px.scatter(dfshow, x='A', y='value', color='component')

## Part 2: compute multiple random particle assignments

### LJ potential

In [106]:
np.random.seed(42)

In [107]:
n_random = 50
list_results = []

In [109]:
%%time

for elem in dfelem.index[1:94]:
    ti = datetime.now()
    sumbol, N, Z, A = dfelem.loc[elem]

    # read coordinates    
    X = read_coords(A)

    print(elem)
    for i in range(n_random):
        T = generate_random_labels(N, Z)

        # compute energy, elmag and strong contribution
        E, Ee, Es = total_energy(X, T)
        epn = E / A
        # print(elem, N, A, E, epn)
        
        list_results.append(
            {'element': elem, 'id': i, 'N': N, 'A': A, 'E': E, 'Ee': Ee, 'Es': Es, 'epn': epn}
        )

Helium
Lithium
Beryllium
Boron
Carbon
Nitrogen
Oxygen
Fluorine
Neon
Sodium
Magnesium
Aluminum
Silicon
Phosphorus
Sulfur
Chlorine
Argon
Potassium
Calcium
Scandium
Titanium
Vanadium
Chromium
Manganese
Iron
Cobalt
Nickel
Copper
Zinc
Gallium
Germanium
Arsenic
Selenium
Bromine
Krypton
Rubidium
Strontium
Yttrium
Zirconium
Niobium
Molybdenum
Technetium
Ruthenium
Rhodium
Palladium
Silver
Cadmium
Indium
Tin
Antimony
Tellurium
Iodine
Xenon
Cesium
Barium
Lanthanum
Cerium
Praseodymium
Neodymium
Promethium
Samarium
Europium
Gadolinium
Terbium
Dysprosium
Holmium
Erbium
Thulium
Ytterbium
Lutetium
Hafnium
Tantalum
Tungsten
Rhenium
Osmium
Iridium
Platinum
Gold
Mercury
Thallium
Lead
Bismuth
Polonium
Astatine
Radon
Francium
Radium
Actinium
Thorium
Protactinium
Uranium
Neptunium
Plutonium
CPU times: user 4min 36s, sys: 3.07 s, total: 4min 39s
Wall time: 4min 54s


### Process and visualise data

In [110]:
dfres = pd.DataFrame(list_results)
dfres['e_elmag_pn'] = dfres['Ee'] / dfres['A']
dfres['e_strong_pn'] = dfres['Es'] / dfres['A']
dfres.head()

,element,id,N,A,E,Ee,Es,epn,e_elmag_pn,e_strong_pn
0,Helium,0,2,4,-13.117148,1.282895,-14.400043,-3.279287,0.320724,-3.600011
1,Helium,1,2,4,-13.117148,1.282895,-14.400043,-3.279287,0.320724,-3.600011
2,Helium,2,2,4,-13.117148,1.282895,-14.400043,-3.279287,0.320724,-3.600011
3,Helium,3,2,4,-13.117146,1.282897,-14.400043,-3.279286,0.320724,-3.600011
4,Helium,4,2,4,-13.117148,1.282895,-14.400043,-3.279287,0.320724,-3.600011


In [112]:
# reshape the dataframe for showing components
dfshow = dfres.melt(id_vars=['id', 'A'], value_vars=['epn', 'e_elmag_pn', 'e_strong_pn'])
dfshow = dfshow.rename({'variable': 'component', 'value': 'value'}, axis=1)
dfshow.tail()

,id,A,component,value
13945,45,244,e_strong_pn,-15.11688
13946,46,244,e_strong_pn,-15.11688
13947,47,244,e_strong_pn,-15.11688
13948,48,244,e_strong_pn,-15.11688
13949,49,244,e_strong_pn,-15.11688


In [114]:
px.scatter(dfres, x='A', y='epn', title='Energy per nucleon vs number of nucleons', range_y=[-10, 0])

In [115]:
px.scatter(dfshow, x='A', y='value', color='component')

In [116]:
# save results
# dfres.to_csv('run_lj_sampling.csv', index=False)

### Morse potential

In [ ]:
np.random.seed(42)

In [ ]:
n_random = 50
list_results = []

In [ ]:
%%time

for elem in dfelem.index[1:94]:
    ti = datetime.now()
    symbol, N, Z, A = dfelem.loc[elem]

    # read coordinates    
    X = read_coords(A)

    print(elem)
    for i in range(n_random):
        T = generate_random_labels(N, Z)

        # compute energy, elmag and strong contribution
        E, Ee, Es = total_energy(X, T, pot='morse')
        epn = E / A
        # print(elem, N, A, E, epn)
    
        list_results.append(
            {'element': elem, 'id': i, 'N': N, 'A': A, 'E': E, 'Ee': Ee, 'Es': Es, 'epn': epn}
        )

Helium
Helium 2 4 -11.525200039372333 -2.881300009843083
Helium 2 4 -11.525200039372333 -2.881300009843083
Helium 2 4 -11.525200464081568 -2.881300116020392
Helium 2 4 -11.525198054264884 -2.881299513566221
Helium 2 4 -11.525200464081568 -2.881300116020392
Helium 2 4 -11.525198054264884 -2.881299513566221
Helium 2 4 -11.525200246109241 -2.8813000615273103
Helium 2 4 -11.525200039372333 -2.881300009843083
Helium 2 4 -11.525200246109241 -2.8813000615273103
Helium 2 4 -11.525200039372333 -2.881300009843083
Helium 2 4 -11.52520184951096 -2.88130046237774
Helium 2 4 -11.525198054264884 -2.881299513566221
Helium 2 4 -11.52520184951096 -2.88130046237774
Helium 2 4 -11.52520184951096 -2.88130046237774
Helium 2 4 -11.525198054264884 -2.881299513566221
Helium 2 4 -11.525200039372333 -2.881300009843083
Helium 2 4 -11.52520184951096 -2.88130046237774
Helium 2 4 -11.52520184951096 -2.88130046237774
Helium 2 4 -11.525200246109241 -2.8813000615273103
Helium 2 4 -11.52520184951096 -2.88130046237774
He

### Process and visualise data

In [ ]:
dfres = pd.DataFrame(list_results)
dfres['e_elmag_pn'] = dfres['Ee'] / dfres['A']
dfres['e_strong_pn'] = dfres['Es'] / dfres['A']
dfres.head()

,element,id,N,A,E,Ee,Es,epn,e_elmag_pn,e_strong_pn
0,Helium,0,2,4,-11.525200,1.282895,-12.808095,-2.8813,0.320724,-3.202024
1,Helium,1,2,4,-11.525200,1.282895,-12.808095,-2.8813,0.320724,-3.202024
2,Helium,2,2,4,-11.525200,1.282895,-12.808095,-2.8813,0.320724,-3.202024
3,Helium,3,2,4,-11.525198,1.282897,-12.808095,-2.8813,0.320724,-3.202024
4,Helium,4,2,4,-11.525200,1.282895,-12.808095,-2.8813,0.320724,-3.202024


In [ ]:
# reshape the dataframe for showing components
dfshow = dfres.melt(id_vars=['id', 'A'], value_vars=['epn', 'e_elmag_pn', 'e_strong_pn'])
dfshow = dfshow.rename({'variable': 'component', 'value': 'value'}, axis=1)
dfshow.head()

,id,A,component,value
0,0,4,epn,-2.8813
1,1,4,epn,-2.8813
2,2,4,epn,-2.8813
3,3,4,epn,-2.8813
4,4,4,epn,-2.8813


In [ ]:
px.scatter(dfres, x='A', y='epn', title='Energy per nucleon vs number of nucleons')

In [ ]:
px.scatter(dfshow, x='A', y='value', color='component')

## Part 3: equilibrate before computing energies

In [ ]:
from scipy.optimize import basinhopping

In [ ]:
# basinhopping?

### Sample minimisation

Results:
- C: raw -62, minimised -102, time 6s
- O: raw -90, minimised -152, time 35s
- Na: raw -138, minimised -231, time 41s
- Al: raw -154 to -159, minimised -264, time 1min44s

In [ ]:
def print_fun(x, f, accepted):
    print("at minimum %.4f accepted %d" % (f, int(accepted)))

In [ ]:
# trial atom
elem = 'Aluminum'

symbol, N, Z, A = dfelem.loc[elem]

# read coordinates   
X = read_coords(A)
T = generate_random_labels(N, Z)

In [ ]:
e_raw = total_energy(X, T, pot='morse')
e_raw[0]

-154.57213663949403

In [ ]:
%%time

res = basinhopping(
    total_energy_min,
    X.flatten(),
    niter=10,
    minimizer_kwargs=dict(method='L-BFGS-B', args=(T, 'morse', 'bh')),
    callback=print_fun,
    niter_success=1
)

at minimum -263.5759 accepted 1
at minimum -241.4545 accepted 0
at minimum -263.7007 accepted 0
CPU times: user 1min 30s, sys: 1.18 s, total: 1min 32s
Wall time: 1min 44s


In [ ]:
res.fun

-263.57587197788837

In [ ]:
res

                    message: ['success condition satisfied']
                    success: True
                        fun: -263.57587197788837
                          x: [-6.882e-01  5.511e-01 ...  8.996e-02
                               4.142e-01]
                        nit: 2
      minimization_failures: 1
                       nfev: 30258
                       njev: 369
 lowest_optimization_result:  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
                              success: True
                               status: 0
                                  fun: -263.57587197788837
                                    x: [-6.882e-01  5.511e-01 ...
                                         8.996e-02  4.142e-01]
                                  nit: 65
                                  jac: [ 1.819e-04  1.876e-03 ...
                                        -4.275e-03 -6.150e-03]
                                 nfev: 6068
                                 njev: 74


### Morse potential using basin hopping

In [ ]:
sample_elems = [
    'Helium',
    'Carbon',
    'Fluorine',
    'Aluminum',
    'Chlorine',
    'Calcium',
    'Chromium',
    'Iron',
    'Nickel',
    'Gallium',
    'Bromine',
    'Zirconium',
    'Rhodium',
    'Indium',
]

In [ ]:
list_results = []
list_coords = []

In [ ]:
%%time

for elem in sample_elems:
    np.random.seed(42)

    sumbol, N, Z, A = dfelem.loc[elem]

    # read coordinates   
    X = read_coords(A)
    T = generate_random_labels(N, Z)

    # compute energy and shortest distance
    ti = datetime.now()
    E = total_energy(X, T)
    print('\n', elem, N, A, E)

    # equilibrate a with basin hopping
    print('Equilibrating...')
    res = basinhopping(
        total_energy_min,
        X.flatten(),
        niter=10,
        minimizer_kwargs=dict(method='L-BFGS-B', args=(T, 'morse', 'bh')),
        callback=print_fun,
        niter_success=1
    )
    Efin = res.fun
    print(elem, N, A, Efin)
    list_results.append(
        {'element': elem, 'id': i, 'N': N, 'A': A, 'E': E, 'Ee': Ee, 'Es': Es, 'epn': epn, 'Efin': Efin}
    )
    list_coords.append({'element': elem, 'coords': res.x})


 Helium 2 4 (-14.467151801962656, 1.2828954036899443, -15.7500472056526)
Equilibrating...
at minimum -19.5624 accepted 1
at minimum -19.5624 accepted 1
at minimum -19.5624 accepted 1
at minimum -10.4984 accepted 0
Helium 2 4 -19.562360536858368

 Carbon 6 12 (-84.73930866578618, 14.925514607601247, -99.66482327338743)
Equilibrating...
at minimum -106.3605 accepted 1
at minimum -93.9288 accepted 0
at minimum -97.1199 accepted 0
Carbon 6 12 -106.36045342051753

 Fluorine 9 19 (-157.23587452617247, 33.496899409031094, -190.73277393520357)
Equilibrating...
at minimum -189.7245 accepted 1
at minimum -173.7489 accepted 0
at minimum -181.9765 accepted 0
Fluorine 9 19 -189.72446316916603

 Aluminum 13 27 (-229.55054775559898, 66.74223690747328, -296.29278466307227)
Equilibrating...
at minimum -266.0244 accepted 1
at minimum -246.8431 accepted 0
at minimum -252.8931 accepted 0
Aluminum 13 27 -266.0244299990546

 Chlorine 17 35 (-302.131086925574, 106.72967855551447, -408.86076548108844)
Equili

In [ ]:
dfres = pd.DataFrame(list_results)
dfres['e_elmag_pn'] = dfres['Ee'] / dfres['A']
dfres['e_strong_pn'] = dfres['Es'] / dfres['A']
dfres['epn_min'] = dfres['Efin'] / dfres['A']
dfres.head()

,element,id,N,A,E,Ee,Es,epn,Efin,e_elmag_pn,e_strong_pn,epn_min
0,Helium,49,2,4,"(-14.467151801962656, 1.2828954036899443, -15....",667.406263,-1312.53869,-5.421281,-19.562361,166.851566,-328.134672,-4.890590
1,Carbon,49,6,12,"(-84.73930866578618, 14.925514607601247, -99.6...",667.406263,-1312.53869,-5.421281,-106.360453,55.617189,-109.378224,-8.863371
2,Fluorine,49,9,19,"(-157.23587452617247, 33.496899409031094, -190...",667.406263,-1312.53869,-5.421281,-189.724463,35.126645,-69.080984,-9.985498
3,Aluminum,49,13,27,"(-229.55054775559898, 66.74223690747328, -296....",667.406263,-1312.53869,-5.421281,-266.024430,24.718750,-48.612544,-9.852757
4,Chlorine,49,17,35,"(-302.131086925574, 106.72967855551447, -408.8...",667.406263,-1312.53869,-5.421281,-350.500040,19.068750,-37.501105,-10.014287


In [ ]:
# reshape the dataframe for showing components
dfshow = dfres.melt(id_vars=['id', 'A'], value_vars=['epn_min', 'e_elmag_pn', 'e_strong_pn'])
dfshow = dfshow.rename({'variable': 'component', 'value': 'value'}, axis=1)
dfshow.head()

,id,A,component,value
0,49,4,epn_min,-4.890590
1,49,12,epn_min,-8.863371
2,49,19,epn_min,-9.985498
3,49,27,epn_min,-9.852757
4,49,35,epn_min,-10.014287


In [ ]:
px.scatter(dfres, x='A', y='epn_min', title='Minimised energy per nucleon vs number of nucleons')

In [ ]:
dfres.to_csv('run_min_bh_selection.csv', index=False)
pd.DataFrame(list_coords).to_csv('coords_min_bh_selection.csv', index=False)